In [20]:
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv

In [21]:
load_dotenv()

True

In [22]:
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

In [23]:
connect = psycopg2.connect(database=DB_NAME,
                           user=DB_USER,
                           password=DB_PASS,
                           host=DB_HOST,
                           port=DB_PORT)

print('База данных успешно открыта')

База данных успешно открыта


In [16]:
cursor = connect.cursor()

In [6]:
cursor.execute("SELECT table_name FROM information_schema.tables \
             WHERE table_schema NOT IN ('information_schema', 'pg_catalog') \
             AND table_schema IN('public', 'myschema');")
cursor.fetchall()

[('building',), ('train_building',)]

In [7]:
building_20230808 = pd.read_csv("../row_data/building_20230808.csv")
building_20230808.head()

,id,full_address
0,56343,"город Пушкин, Кедринская улица, дом 12"
1,595,"поселок Ушково, Пляжевая улица, дом 4Б"
2,7134,"г.Санкт-Петербург, проспект Маршала Жукова, до..."
3,124415,"поселок Белоостров, Дюны, Центральная улица, д..."
4,185368,"поселок Песочный, Речная улица, дом 28, литера Б"


In [13]:
cursor.execute("DROP TABLE IF EXISTS building;")
connect.commit()

In [14]:
cursor.execute("""CREATE TABLE building( 
                row_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, 
                id INTEGER UNIQUE NOT NULL,
                full_address TEXT NOT NULL);""")
connect.commit() 

In [15]:
data = building_20230808.values
data[:4]

array([[56343, 'город Пушкин, Кедринская улица, дом 12'],
       [595, 'поселок Ушково, Пляжевая улица, дом 4Б'],
       [7134,
        'г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е'],
       [124415,
        'поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2']],
      dtype=object)

In [16]:
cursor.executemany("""INSERT INTO building( 
                      id, full_address) 
                      VALUES (%s, %s);""", data)
connect.commit()

In [17]:
cursor.execute("SELECT * FROM building LIMIT 10;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,row_id,id,full_address
0,1,56343,"город Пушкин, Кедринская улица, дом 12"
1,2,595,"поселок Ушково, Пляжевая улица, дом 4Б"
2,3,7134,"г.Санкт-Петербург, проспект Маршала Жукова, до..."
3,4,124415,"поселок Белоостров, Дюны, Центральная улица, д..."
4,5,185368,"поселок Песочный, Речная улица, дом 28, литера Б"
5,6,2459,"г.Санкт-Петербург, Большой проспект П.С., дом ..."
6,7,112309,"поселок Солнечное, Комсомольская улица, дом 15..."
7,8,187158,"г.Санкт-Петербург, набережная Обводного канала..."
8,9,16759,"поселок Александровская, Беличья улица, дом 7"
9,10,24236,"поселок Парголово, Осиновая Роща, Вокзальное ш..."


In [18]:
cursor.execute("SELECT COUNT(*) FROM building;")
cursor.fetchone()

(166645,)

In [19]:
len(data)

166645

In [20]:
building_20230808.head()

,id,full_address
0,56343,"город Пушкин, Кедринская улица, дом 12"
1,595,"поселок Ушково, Пляжевая улица, дом 4Б"
2,7134,"г.Санкт-Петербург, проспект Маршала Жукова, до..."
3,124415,"поселок Белоостров, Дюны, Центральная улица, д..."
4,185368,"поселок Песочный, Речная улица, дом 28, литера Б"


In [22]:
data = building_20230808.set_index('id').drop_duplicates().values
data[:5]

array([['город Пушкин, Кедринская улица, дом 12'],
       ['поселок Ушково, Пляжевая улица, дом 4Б'],
       ['г.Санкт-Петербург, проспект Маршала Жукова, дом 30, корпус 2, литера Е'],
       ['поселок Белоостров, Дюны, Центральная улица, дом 5, корпус 2'],
       ['поселок Песочный, Речная улица, дом 28, литера Б']], dtype=object)

In [23]:
cursor.execute("DROP TABLE IF EXISTS train_building;")
connect.commit()

In [24]:
cursor.execute("""CREATE TABLE train_building( 
                row_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, 
                full_address TEXT UNIQUE NOT NULL);""")
connect.commit() 

In [25]:
cursor.executemany("""INSERT INTO train_building( 
                      full_address) 
                      VALUES (%s);""", data)
connect.commit()

In [26]:
cursor.execute("SELECT * FROM train_building LIMIT 10;")
rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=[_.name for _ in cursor.description])
df

,row_id,full_address
0,1,"город Пушкин, Кедринская улица, дом 12"
1,2,"поселок Ушково, Пляжевая улица, дом 4Б"
2,3,"г.Санкт-Петербург, проспект Маршала Жукова, до..."
3,4,"поселок Белоостров, Дюны, Центральная улица, д..."
4,5,"поселок Песочный, Речная улица, дом 28, литера Б"
5,6,"г.Санкт-Петербург, Большой проспект П.С., дом ..."
6,7,"поселок Солнечное, Комсомольская улица, дом 15..."
7,8,"г.Санкт-Петербург, набережная Обводного канала..."
8,9,"поселок Александровская, Беличья улица, дом 7"
9,10,"поселок Парголово, Осиновая Роща, Вокзальное ш..."


In [27]:
cursor.execute("SELECT COUNT(*) FROM train_building;")
cursor.fetchone()

(157734,)

In [28]:
len(data)

157734

In [29]:
cursor.execute("DROP TABLE IF EXISTS query_addrese;")
connect.commit()

In [11]:
cursor.execute("""CREATE TABLE query_addrese( 
                row_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, 
                query_addrese TEXT UNIQUE NOT NULL,
                count INTEGER);""")
connect.commit() 

In [12]:
cursor.execute("DROP TABLE IF EXISTS predict_addrese;")
connect.commit()

In [17]:
cursor.execute("""CREATE TABLE predict_addrese( 
                row_id INTEGER PRIMARY KEY GENERATED BY DEFAULT AS IDENTITY, 
                predict_addrese TEXT UNIQUE NOT NULL,
                count INTEGER);""")
connect.commit() 

In [18]:
cursor.execute("SELECT table_name FROM information_schema.tables \
             WHERE table_schema NOT IN ('information_schema', 'pg_catalog') \
             AND table_schema IN('public', 'myschema');")
cursor.fetchall()

[('building',), ('train_building',), ('query_addrese',), ('predict_addrese',)]

In [24]:
connect.close()